In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from matplotlib import pyplot as plt
import scvelo as scv
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rcParams

In [ ]:
sc.settings.verbosity=3
sc.logging.print_header()
sc.set_figure_params(dpi=300, fontsize=10)
FIGSIZE = (4, 3)
rcParams["figure.figsize"] = FIGSIZE

In [ ]:
oviduct = sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/version2oviduct_concat.h5ad')
oviduct.var_names_make_unique()
oviductepithelial=oviduct[oviduct.obs['Cell_Types'].isin(['Secretory','Ciliated']),:].copy()
oviductepithelial = oviductepithelial[~oviductepithelial.obs['Patient'].isnull()]
oviduct

In [ ]:
menopausal_dict = { 'Proliferative':['patient 1','patient 4','patient 6'],
            'Secretory':['patient 3','patient 5','patient 7','patient 9','patient 12'],
            'Peri-menopausal':['patient 4','patient 10','patient 11'],
            'Post-menopausal':['patient 8','patient 13','patient 15'],}

oviductepithelial.obs['menstrual_status'] = np.nan
for i in menopausal_dict.keys():
    ind = pd.Series(oviductepithelial.obs.Patient).isin(menopausal_dict[i])
    oviductepithelial.obs.loc[ind,'menstrual_status'] = i

oviductepithelial.obs['menstrual_status']= oviductepithelial.obs['menstrual_status'].astype('category')
oviductepithelial.obs['menstrual_status']

In [ ]:
age_dict = { 47:['patient 1','patient 3'], 41:['patient 4'], 33:['patient 5'],
            31:['patient 6'],
            46:['patient 7','patient 9'],
            62:['patient 8'],
            52:['patient 10'],
            50:['patient 11'],
            45:['patient 12'],
            64:['patient 13'],
            55:['patient 15'],}

oviductepithelial.obs['Age'] = np.nan
for i in age_dict.keys():
    ind = pd.Series(oviductepithelial.obs.Patient).isin(age_dict[i])
    oviductepithelial.obs.loc[ind,'Age'] = i

oviductepithelial.obs['Age']= oviductepithelial.obs['Age'].astype('category')
oviductepithelial.obs['Age']

In [ ]:
cycleday_dict = {'day 3':['patient 1'],
            'day 17':['patient 3'],
            'day 7':['patient 4'],
            'day 31':['patient 5'],
            'day 13':['patient 6'],
            'day 23':['patient 7'],
            'day 28':['patient 9'],
            'day 37':['patient 10'], 
            'unknown':['patient 11','patient 12'],
            'post-menopausal':['patient 8','patient 13','patient 15']}

oviductepithelial.obs['cycle_day'] = np.nan
for i in cycleday_dict.keys():
    ind = pd.Series(oviductepithelial.obs.Patient).isin(cycleday_dict[i])
    oviductepithelial.obs.loc[ind,'cycle_day'] = i

oviductepithelial.obs['cycle_day']= oviductepithelial.obs['cycle_day'].astype('category')
oviductepithelial.obs['cycle_day']

In [ ]:
tissue_dict ={'Fallopian tube - Ampulla':['Fallopian tube Ampulla cells','Fallopian tube (Ampulla)'],
              'Fallopian tube - Fimbriae':['Fallopian tube Fimbria cells','Fallopian tube (Fimbria)'],
              'Fallopian tube - Isthmus':['Fallopian tube Isthmus cells','Fallopian tube (Isthmus)'],
              'Fallopian tube - Infundibulum':['normal fallopian tube cells','Fallopian tube (Infundibulum)'],
              'Fallopian tube - Unspecified':['Fallopian tube','Left fallopian tube','Right fallopian tube']}

oviductepithelial.obs['Tissuelocation'] = np.nan
for i in tissue_dict.keys():
    ind = pd.Series(oviductepithelial.obs.Tissue).isin(tissue_dict[i])
    oviductepithelial.obs.loc[ind,'Tissuelocation'] = i

oviductepithelial.obs['Tissuelocation']= oviductepithelial.obs['Tissuelocation'].astype('category')
oviductepithelial.obs['Tissuelocation']

oviductepithelial

In [ ]:
sc.pp.combat(oviductepithelial, key='Patient')
sc.tl.pca(oviductepithelial, svd_solver='arpack', n_comps=150)
sc.pl.pca_variance_ratio(oviductepithelial)

Plot UMAPS

In [ ]:
sc.pp.neighbors(oviductepithelial, n_neighbors=90,n_pcs=7)
sc.tl.leiden(oviductepithelial,resolution=0.8)
sc.tl.umap(oviductepithelial)
sc.pl.umap(oviductepithelial, color=['Author','leiden','Cell_Types','OVGP1','PAX8','LGR5','SOX9','PAEP'])
scv.pp.moments(oviductepithelial)
scv.tl.velocity(oviductepithelial, fit_offset2=True)
scv.tl.velocity_graph(oviductepithelial)
scv.pl.velocity_embedding_stream(oviductepithelial, basis='umap', color='leiden',
                                 legend_fontsize='6', legend_loc='right margin')
scv.pl.velocity_embedding_stream(oviductepithelial, basis='umap', color='LGR5',
                                 legend_fontsize='6', legend_loc='right margin')

In [ ]:
results_file = '/home/j87832lw/mounting/oviduct/data/oviductepitheliumforconcat.h5ad'
oviductepithelial.write(results_file)

In [ ]:
oviductepithelial=sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/oviductepitheliumforconcat.h5ad')

Label Leiden clusters

In [ ]:
epithelial_celltypes = {'Pre-ciliated-2':['1'],
                        'Progenitor':['2'],
                        'Mature secretory':['3','9','5'],
                        'Mature multiciliated':['4','7','0'],
                        'Pre-ciliated-1':['6'],
                        'OVGP1+ ciliated':['8'],
                       'LGR5+ secretory':['10'],
                        'Mature secretory (Primary cilia)':['11']}

oviductepithelial.obs['epithelial_celltypes'] = np.nan

for i in epithelial_celltypes.keys():
    ind = pd.Series(oviductepithelial.obs.leiden).isin(epithelial_celltypes[i])
    oviductepithelial.obs.loc[ind,'epithelial_celltypes'] = i

oviductepithelial.obs['epithelial_celltypes']= oviductepithelial.obs['epithelial_celltypes'].astype('category')
oviductepithelial.obs['epithelial_celltypes']

sc.pl.umap(oviductepithelial, color=['epithelial_celltypes']

In [ ]:
sc.pl.umap(oviductepithelial, color=['LGR5','OVGP1','FOXJ1'])

In [ ]:
marker_genes= ['TPPP3','FOXJ1','PIFO','CFAP157','C9orf24','SPAG6','OVGP1',
               'CRISP3','PGR','CXCL2','NOTCH2','ATP2B4','C3','LGR5','BCAM',
               'MSLN','VPS4A','PAX8']

sc.tl.dendrogram(oviductepithelial, groupby='epithelial_celltypes')

sc.pl.dotplot(oviductepithelial, marker_genes, groupby='epithelial_celltypes', dendrogram=True, var_group_rotation=[90],swap_axes=True)


Marker gene heatmaps 

In [ ]:
sc.pp.log1p(oviductepithelial)
sc.tl.rank_genes_groups(oviductepithelial,'epithelial_celltypes', method='wilcoxon')
sc.tl.dendrogram(oviductepithelial, groupby='epithelial_celltypes')
sc.pl.rank_genes_groups_heatmap(oviductepithelial, n_genes=25, swap_axes=True, show_gene_labels=False,
                                vmin=-3, vmax=3)

Further velocity analysis 

In [ ]:
scv.tl.velocity_confidence(oviductepithelial)
keys = 'velocity_length','velocity_confidence'
scv.pl.scatter(oviductepithelial, c=keys, cmap='coolwarm', perc=[5,95], size=4) 

In [ ]:
df = oviductepithelial.obs.groupby('epithelial_celltypes')[keys].mean().T
styled_df= df.style.background_gradient(cmap='coolwarm', axis=1)
styled_df=styled_df.format('{:.2f}')

styled_df